# COGS 108 - Data Checkpoint

# Names

- Brandon Luong
- Noel Molina
- Margarita Leonova
- Tatiana Bimatova
- Huy Le

# Updated Project Proposal

# Research Question
What characteristics of beer (flavors like bitter/sweet/sour/spicy etc.; alcohol percentage; color) influence the rating of beer (Plainly speaking, what kind of beer do people like more)?

# Background and Prior Work
Beer is the most popular alcoholic drink in the world. Even if we take into consideration all drinks, beer is second only (!) to tea and water [^wiki].

Beer is an alcoholic beverage created from combining 4 main ingredients: water, malt, hops and yeast. The alcohol content in beer is created through yeast fermentation. Depending on the fermentation process, beer will be classified as ale or lager. Under these categories are more specific styles, such as pale ale, India pale ale (IPA), stouts, etc. [^wiki].

Several beer rating websites show that stouts (namely the ones with higher alcohol percentage of 10% and more) tend to have very high consumer ratings. 

Recent research [^manikas] suggests that higher ABV (alcohol by volume) correlated significantly and positively with higher consumer ratings while the higher length of a review correlated negatively with the rating score. 

Another research [^janish] supports the relationship between a higher ABV and consumer rating.

In a “A genome-wide association study of bitter and sweet beverage consumption” [^zhong], the authors found that people’s “taste preferences for bitter or sweet beverages aren't based on variations in our taste genes, but rather genes related to the psychoactive properties of these beverages”. In other words, people like bitter beverages not for their taste, but for the effect these beverages have on us.

However we were not able to find a research that shows correlation between beer flavor and color characteristics and higher ratings.

-------------------------------------------------------------------------------------------

[^wiki]: Beer. https://en.wikipedia.org/wiki/Beer

[^manikas]: Andrew Manikas, Michael Godfrey and Jason Woldt. (2022) What drives higher beer ratings? Evidence from Big Data. *Econpapers*. https://econpapers.repec.org/article/ibfijmmre/v_3a15_3ay_3a2022_3ai_3a1_3ap_3a1-13.htm

[^janish]: Scott Janish. Relationship of ABV to Beer Scores. *http://scottjanish.com/*. http://scottjanish.com/relationship-of-abv-to-beer-scores/

[^zhong]: Victor W Zhong, Alan Kuang, Rebecca D Danning, Peter Kraft, Rob M van Dam, Daniel I Chasman, Marilyn C Cornelis. (02 May 2019) A genome-wide association study of bitter and sweet beverage consumption. *Human Molecular Genetics*. https://academic.oup.com/hmg/article/28/14/2449/5424254

# Hypothesis
According to the Beer Advocate and Untappd ratings, the majority of the top rated beers are stouts. Therefore, we hypothesize that characteristics like higher alcohol percentage (10% and more), bitter-sweet taste and dark color contribute the most to the higher rating.

Sources:

https://www.beeradvocate.com/beer/top-rated/us/

https://untappd.com/beer/top_rated

# Data
Our ideal dataset would include beer name, its type and rating. We also want characteristics of the beer, such as ABV, flavor characteristics, color.

We think that internet scraping won’t be suitable for our purposes. We want a dataset that comes from a large poll or a rating resource like Untappd or BeerAdvocate. These resources provide hundreds of customer reviews as well as beer characteristics.

Our observation would be a single customer review with a beer rating. We want the most recent data we can find because we’re not much interested in what people liked a long time ago.


Here are some real datasets we could potentially use:

https://www.kaggle.com/datasets/jailenbailey/untappd-beer-data?resource=download (Untappd beer data)

https://www.kaggle.com/datasets/rdoume/beerreviews (Beer reviews)


Untappd beer data contain beer name, style, and brewing company. Each beer listed has their alcohol content and measure of bitterness displayed. Additionally, there are comments from consumers detailing what they like about a specific beer or what their overall review was.


Beer reviews contain the type of beer (labeled as beer style) and its brewing company. There are numeric ratings (on a scale of 1-5) for beer aroma, taste, appearance, and overall final score of beer. 


These datasets contain both beer style and brewing company. However, Untapped beer data gives qualitative information regarding beer likes as it contains comments from consumers about the beer they tasted. Meanwhile, Beer reviews contain quantitative data on beer characteristics as consumers rated aroma, taste, and appearance on a numeric scale. 


# Ethics & Privacy
Our potential dataset (Untappd and BeerAdvocates) does not include any personal identifiable information. If we collect more data with personally identifiable information, we will disregard the sensitive information, such as a person’s name, in our report. It does contain anonymous comments about the beer. Since these comments are anonymous, their privacy is protected. 

Both of the companies the data is coming from are not affiliated with any beer companies, so we don’t have any evidence of them being biased.

We will make inferences from the results obtained from the experiment and not based on our hypothesis.

Beer is an alcoholic beverage. So, to purchase it, individuals are verified to ensure they are of age. Our dataset gathers information from individuals that are legally allowed to purchase beer. 


# Team Expectations
- Communicate through discord and zoom (messaging) and during in person group meetings
- Divide up work equally and do one’s part 
- Make regular check ups on work progress 
- Make sure that each member has an understanding of the project and section
- Reach out to group members, TAs, and professor if you need any help
- If any members are uncooperative try to reach out and discuss possible solutions


# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 10/14  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 10/21  |  12 PM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 10/24  | 12 PM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 11/4  | 12 PM  | Import & Wrangle Data; EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 11/18  | 12 PM  | Finalize wrangling/EDA; Begin Analysis | Discuss/edit Analysis; Complete project check-in |
| 12/5  | 12 PM  | Complete analysis; Draft results/conclusion/discussion| Discuss/edit full project |
| 12/9  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |

#


# Data Checkpoint

<a id='research_question'></a>
# Research Question

What characteristics of beer (flavors like bitter/sweet/sour/spicy etc.; alcohol percentage; color) influence the rating of beer (Plainly speaking, what kind of beer do people like more)?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION